In [3]:
from pyhealth.datasets import UMLSDataset

umls_ds = UMLSDataset(
    root="https://storage.googleapis.com/pyhealth/umls/",
    dev=True,
    refresh_cache=False
)

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
umls_ds.stat()


Statistics of base dataset (dev=True):
	- Dataset: UMLSDataset
	- Number of triples: 88176
	- Number of entities: 9737
	- Number of relations: 8
	- Task name: Null
	- Number of samples: 0



In [5]:
from pyhealth.tasks import link_prediction_fn

umls_ds = umls_ds.set_task(link_prediction_fn, negative_sampling=256)

Loading UMLSDataset base dataset from /home/pj20/.cache/pyhealth/datasets/46e7370273967c215741135e6ccdd2b9.pkl


In [6]:
umls_ds.stat()


Statistics of base dataset (dev=True):
	- Dataset: UMLSDataset
	- Number of triples: 88176
	- Number of entities: 9737
	- Number of relations: 8
	- Task name: link_prediction_fn
	- Number of samples: 176352



In [15]:
from pyhealth.datasets import split_by_keys, get_dataloader_kg

train_dataset, val_dataset, test_dataset = split_by_keys(umls_ds, [0.9, 0.05, 0.05])

In [16]:
len(train_dataset['head_test'])

79358

In [26]:
batch_size = 256

train_loader, num_batch_train = get_dataloader_kg(train_dataset, batch_size=batch_size, train=True, shuffle=True)
val_loader, num_batch_val = get_dataloader_kg(val_dataset, batch_size=batch_size, train=False, shuffle=False)
test_loader, num_batch_test = get_dataloader_kg(test_dataset, batch_size=batch_size, train=False, shuffle=False)

In [27]:
from pyhealth.models.kg import TransE, RotatE

model = RotatE(
    dataset=umls_ds
)

In [ ]:
from pyhealth.trainer import Trainer
import torch

trainer = Trainer(model=model, device='cuda')
trainer.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=20,
    batch_size=batch_size,
    ## num_batch=num_batch_train,
    steps_per_epoch=100,
    optimizer_params={
        'lr': 1e-3
    }
)

In [6]:
tmp = [(0, rand_head) for rand_head in range(9700)]

In [5]:
import torch 

head = 2
gt_head = [0, 2, 9698]
tmp = torch.LongTensor([any_head if any_head not in gt_head else head for any_head in range(9700)])

In [10]:
tmp

tensor([   2,    1,    2,  ..., 9697,    2, 9699])

In [8]:
torch.LongTensor(tmp)[:, 1]

tensor([   0,    1,    2,  ..., 9697, 9698, 9699])

In [6]:
import numpy as np

bias_head = np.arange(0, 9700)
bias_head[gt_head] = head

In [17]:
import numpy as np

y_prob_all = []
y_prob_all.append([[0,1,2]])
y_prob_all.append([[4,2,5]])

y_prob_all = np.concatenate(y_prob_all, axis=0)

In [19]:
import torch 

argsort = torch.argsort(torch.Tensor(y_prob_all), dim=1, descending=True)

In [33]:
y_true = [1, 2]
argsort = np.argsort(-y_prob_all, axis=1)

In [53]:
np.array([np.where(argsort[i] == y_true[i])[0][0] for i in range(len(y_true))]) + 1

array([2, 1])

In [43]:
argsort

array([[2, 1, 0],
       [2, 0, 1]])

In [61]:
ranking = np.array([11, 2, 4, 9, 7, 12, 5, 4])

np.mean(1 / ranking)

0.20352633477633478